## Imports

In [1]:
import numpy as np 
import os
import csv
import torch
import matplotlib.pyplot as plt
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from skimage.measure import shannon_entropy
import cv2
import piq
#I AM LAZY AND IF IT SAYS SIGMA = 20 IT MEANS 25

## Gets the images organized

In [2]:

# Start from current directory
current_dir = os.getcwd()
target_folder = "Noise_Reduction_CS506"
found = False

print(f"Starting from: {current_dir}")

# Keep going up until we find the target folder
while True:
    # Check if target folder exists in current directory
    potential_path = os.path.join(current_dir, target_folder)
    
    if os.path.exists(potential_path):
        print(f"✅ Found '{target_folder}' at: {potential_path}")
        found = True
        
        # Change to that directory
        os.chdir(potential_path)
        print(f"Changed to: {os.getcwd()}")
        break
    
    # If we're at the root, stop
    if current_dir == os.path.dirname(current_dir):  # At root
        print(f"❌ Could not find '{target_folder}' in any parent directory")
        print(f"Reached root: {current_dir}")
        break
    
    # Go up one level
    current_dir = os.path.dirname(current_dir)
    print(f"Going up to: {current_dir}")

if not found:
    print("\nOptions:")
    print("1. Make sure 'Noise_Reduction_CS506' folder exists somewhere above")
    print("2. Run the script from inside or near that folder")
    print("3. Manually navigate: cd /path/to/Noise_Reduction_CS506")

Starting from: /Users/zevfine/Desktop/Noise_Reduction_CS506/Data_Results/Data
Going up to: /Users/zevfine/Desktop/Noise_Reduction_CS506/Data_Results
Going up to: /Users/zevfine/Desktop/Noise_Reduction_CS506
Going up to: /Users/zevfine/Desktop
✅ Found 'Noise_Reduction_CS506' at: /Users/zevfine/Desktop/Noise_Reduction_CS506
Changed to: /Users/zevfine/Desktop/Noise_Reduction_CS506


In [3]:
original_folder_path = './Photos_Subset/Original'
BW = "./Photos_Subset/Noised_Images/B&W"
Color = "./Photos_Subset/Noised_Images/Color"

#BW
BW_0_20 = "./Photos_Subset/Noised_Images/BW/mu_equals_zero/25"
BW_0_50 = "./Photos_Subset/Noised_Images/BW/mu_equals_zero/50"
BW_0_100 = "./Photos_Subset/Noised_Images/BW/mu_equals_zero/100"
BW_0_150 = "./Photos_Subset/Noised_Images/BW/mu_equals_zero/150"
BW_neg_20 = "./Photos_Subset/Noised_Images/BW/mu_equals_neg/25"
BW_neg_50 = "./Photos_Subset/Noised_Images/BW/mu_equals_neg/50"
BW_neg_100 = "./Photos_Subset/Noised_Images/BW/mu_equals_neg/100"
BW_neg_150 = "./Photos_Subset/Noised_Images/BW/mu_equals_neg/150"
BW_plus_20 = "./Photos_Subset/Noised_Images/BW/mu_equals_pos/25"
BW_plus_50 = "./Photos_Subset/Noised_Images/BW/mu_equals_pos/50"
BW_plus_100 = "./Photos_Subset/Noised_Images/BW/mu_equals_pos/100"
BW_plus_150 = "./Photos_Subset/Noised_Images/BW/mu_equals_pos/150"


#Color
Color_0_20 = "./Photos_Subset/Noised_Images/Color/mu_equals_zero/25"
Color_0_50 = "./Photos_Subset/Noised_Images/Color/mu_equals_zero/50"
Color_0_150 = "./Photos_Subset/Noised_Images/Color/mu_equals_zero/100"
Color_0_100 = "./Photos_Subset/Noised_Images/Color/mu_equals_zero/150"
Color_neg_20 = "./Photos_Subset/Noised_Images/Color/mu_equals_neg/25"
Color_neg_50 = "./Photos_Subset/Noised_Images/Color/mu_equals_neg/50"
Color_neg_100 = "./Photos_Subset/Noised_Images/Color/mu_equals_neg/100"
Color_neg_150 = "./Photos_Subset/Noised_Images/Color/mu_equals_neg/150"
Color_plus_20 = "./Photos_Subset/Noised_Images/Color/mu_equals_pos/25"
Color_plus_50 = "./Photos_Subset/Noised_Images/Color/mu_equals_pos/50"
Color_plus_100 = "./Photos_Subset/Noised_Images/Color/mu_equals_pos/100"
Color_plus_150 = "./Photos_Subset/Noised_Images/Color/mu_equals_pos/150"


#Creates a list
BW_folders = [BW_0_20, BW_0_50, BW_0_100, BW_0_150, BW_neg_20, BW_neg_50, BW_neg_100, BW_neg_150, BW_plus_20, BW_plus_50, BW_plus_100, BW_plus_150]
Color_folders = [Color_0_20, Color_0_50, Color_0_100, Color_0_150, Color_neg_20, Color_neg_50, Color_neg_100, Color_neg_150, Color_plus_20,Color_plus_50, Color_plus_100, Color_plus_150]

#Gets the BW and Color pairs of the original #bw is in [-6:-4]
dict_pics = set() # i know its a set not a dictionary, but this is funnier
tuple_list = []
#creates a list of tuples of each image (color, bw, tuple)
files = list(os.listdir(original_folder_path))
files.sort()
for filename in files:
    if filename in dict_pics:
        continue
    if(filename[-6:-4] == "bw"):
        temp = filename[:-7] + filename[-4:]
        noise_temp = "noisy_" + temp
        noise_temp_bw = "noisy_" + filename
        dict_pics.add(filename) 
        dict_pics.add(temp)
        tuple_list.append((temp, filename, noise_temp, noise_temp_bw))
    else:
        temp = filename[:-4] + "_bw" + filename[-4:]
        noise_temp = "noisy_" + filename
        noise_temp_bw = "noisy_" + temp
        dict_pics.add(filename) 
        dict_pics.add(temp)
        tuple_list.append((filename, temp, noise_temp, noise_temp_bw))

print(tuple_list[1])

('BSDS300_11_66039.jpg', 'BSDS300_11_66039_bw.jpg', 'noisy_BSDS300_11_66039.jpg', 'noisy_BSDS300_11_66039_bw.jpg')


##

## Image Analysis on Original Photos

In [4]:
rows = []
for i in tuple_list:
    img_path = original_folder_path + "/" + i[0]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Calculate metrics (all direct from image)
    avg_brightness = np.mean(img)
    contrast = np.std(img)
    entropy = shannon_entropy(img)
    
    
    # Sharpness
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    
    # Spatial Frequency
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    dynamic_range = np.ptp(img) / 255.0

    #creates row for csv file
    rows.append({"Name": i[0], "Average Brightness": avg_brightness, "Contrast": contrast, "Entropy": entropy, "Sharpness": sharpness, "Spatial Frequency": spatial_freq, "Dynamic Range": dynamic_range})
#creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Original_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


## Data for BW

In [5]:
rows = []
for i in tuple_list:
    img_path = original_folder_path + "/" + i[1]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   
    # Calculate metrics (all direct from image)
    avg_brightness = np.mean(img)
    contrast = np.std(img)
    entropy = shannon_entropy(img)
    
    
    # Sharpness
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    
    # Spatial Frequency
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    dynamic_range = np.ptp(img) / 255.0

    #creates row for csv file
    rows.append({"Name":i[1], "Average Brightness": avg_brightness, "Contrast": contrast, "Entropy": entropy, "Sharpness": sharpness, "Spatial Frequency": spatial_freq, "Dynamic Range": dynamic_range})
#creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "BW_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile: 
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


## CSV For MU = 0

Sigma = 25

In [6]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_0_20 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
# ... [all your previous code] ...

# Creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,25)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [7]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_0_20 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
    
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,25)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


Sigma = 50

In [8]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_0_50 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,50)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [9]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_0_50 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,50)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


Sigma = 100

In [10]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_0_100 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,100)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [11]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_0_100 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,100)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

Sigma = 150

In [12]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_0_150 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file

csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,150)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

BW

In [13]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_0_150 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(0,150)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


## CSV For MU = 10

Simga = 25

In [14]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_plus_20 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,25)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [15]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_plus_20 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,25)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)



Sigma = 50

In [16]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_plus_50 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,50)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [17]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_plus_50 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,50)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


Sigma = 100

In [18]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_plus_100 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,100)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

In [19]:
BW

'./Photos_Subset/Noised_Images/B&W'

In [20]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_plus_100 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,100)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

Sigma = 150

In [21]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_plus_150 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,150)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [22]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_plus_20 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(10,150)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

## CSV For MU = -10

Simga = 25

In [23]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_neg_20 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(-10,25)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [24]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_neg_20 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(-10,25)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


Sigma = 50

In [25]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_neg_50 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
os.makedirs(csv_dir, exist_ok=True)
csv_path = os.path.join(csv_dir, "Noisy_N(-10,50)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [26]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_neg_50 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
csv_path = os.path.join(csv_dir, "Noisy_N(-10,50)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


Sigma = 100

In [27]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_neg_100 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
csv_path = os.path.join(csv_dir, "Noisy_N(-10,100)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [28]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_neg_100 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
csv_path = os.path.join(csv_dir, "Noisy_N(-10,100)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


Sigma = 150

In [29]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[0]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = Color_neg_150 + "/" + i[2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
csv_path = os.path.join(csv_dir, "Noisy_N(-10,150)_Photo_Data.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)


BW

In [30]:
rows = []
for i in tuple_list:
    original_img_path = original_folder_path + "/" + i[1]
    original_img = cv2.imread(original_img_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_path = BW_neg_150 + "/" + i[3]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Calculate metrics (all direct from image)
    psnr = peak_signal_noise_ratio(original_img, img)
    ssim = structural_similarity(original_img, img, channel_axis=-1)
    mse = mean_squared_error(original_img, img)
    entropy_diff = shannon_entropy(img) - shannon_entropy(original_img)
    
    # Convert to grayscale for luminance-based metrics
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    noise_variance = np.var(img_gray)  # Grayscale is better for noise
    
    # Frequency analysis (on color)
    fft = np.fft.fft2(img, axes=(0, 1))
    spatial_freq = np.mean(np.abs(fft))
    
    # Dynamic range (on color)
    dynamic_range = np.ptp(img) / 255.0
    
    #creates row for csv file
    rows.append({"Name":i[2], "PSNR": psnr, "SSIM": ssim, "MSE": mse, "Entropy_Diff": entropy_diff, 
                 "Noise_Variance": noise_variance, "Sharpness": sharpness, "Spatial_Freq": spatial_freq, 
                 "Dynamic_Range": dynamic_range})
    #creates csv file
csv_dir = "./Data_Results/Data/noised_images_data/gauss"
csv_path = os.path.join(csv_dir, "Noisy_N(-10,150)_Photo_Data_BW.csv")
with open(csv_path, mode = "w") as csvfile:  # <-- USE csv_path HERE, NOT the plain filename
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)
